In [ ]:
import rasterio
import geemap
import ee

Map = geemap.Map()
Map

In [ ]:
roi = geemap.geojson_to_ee(r'E:/Data/GIS/JX.json')
Map.addLayer(roi, {}, 'JX')
Map.center_object(roi)

In [ ]:
def clip(img):
    return img.clip(roi.geometry())
def modis_NDVI(img):
    img1 = img.clip(roi.geometry())
    img2 = img1.normalizedDifference(['sur_refl_b02', 'sur_refl_b01'])
    return img2

In [ ]:
modis = ee.ImageCollection("MODIS/006/MOD09Q1").filterDate('2015', '2016')
landsat8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NDVI')\
            .filterDate('2015', '2016')

In [ ]:
modis_names = modis.aggregate_array('system:id').getInfo()
landsat_names = landsat8.aggregate_array('system:id').getInfo()

In [ ]:
modis_name = modis_names[0]
landsat_name = landsat_names[0]
modis_img = modis_NDVI(ee.Image(modis_name))
landsat_img = clip(ee.Image(landsat_name))
NDVI_img = landsat_img.unmask(modis_img)

In [ ]:
f_name = 'NDVI_'+modis_name.split('/')[-1]
geemap.ee_export_image_to_drive(NDVI_img, description=f_name,
                                folder='Image', scale=30)